In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error,confusion_matrix,accuracy_score,mean_absolute_error
from sklearn.model_selection import cross_val_score, train_test_split,cross_val_score,GridSearchCV,KFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor,BaggingClassifier,\
BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import LabelEncoder,MaxAbsScaler,MinMaxScaler,OneHotEncoder,StandardScaler
from sklearn.linear_model import LinearRegression, Lasso,Ridge,ElasticNet,LogisticRegression

In [2]:
df = pd.read_csv('automobiles.csv')

In [3]:
df.head(3)

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,1970-01-01,US,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,1970-01-01,US,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,1970-01-01,US,plymouth satellite


In [4]:
df['yr'] = pd.to_datetime(df['yr'])
df['year'] = df['yr'].dt.year

In [6]:
df = df.drop('yr', axis=1)

In [7]:
label_enc = LabelEncoder()
df['origin_enc'] = label_enc.fit_transform(df['origin'])

In [8]:
df.head(3)

,mpg,cyl,displ,hp,weight,accel,origin,name,year,origin_enc
0,18.0,8,307.0,130,3504,12.0,US,chevrolet chevelle malibu,1970,2
1,15.0,8,350.0,165,3693,11.5,US,buick skylark 320,1970,2
2,18.0,8,318.0,150,3436,11.0,US,plymouth satellite,1970,2


In [9]:
df = df.drop('origin', axis=1)

In [10]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
one_hot_enc = OneHotEncoder(sparse_output=False)
one_hot_encoded = one_hot_enc.fit_transform(df[categorical_columns])

In [14]:
one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_enc.get_feature_names_out(categorical_columns))
one_hot_df

,name_amc ambassador brougham,name_amc ambassador dpl,name_amc ambassador sst,name_amc concord,name_amc concord d/l,name_amc concord dl 6,name_amc gremlin,name_amc hornet,name_amc hornet sportabout (sw),name_amc matador,...,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df_encoded = pd.concat([df, one_hot_df], axis=1)
df_encoded.head()

,mpg,cyl,displ,hp,weight,accel,name,year,origin_enc,name_amc ambassador brougham,...,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
0,18.0,8,307.0,130,3504,12.0,chevrolet chevelle malibu,1970,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,8,350.0,165,3693,11.5,buick skylark 320,1970,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,18.0,8,318.0,150,3436,11.0,plymouth satellite,1970,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16.0,8,304.0,150,3433,12.0,amc rebel sst,1970,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17.0,8,302.0,140,3449,10.5,ford torino,1970,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#df_encoded = df_encoded.drop(categorical_columns, axis=1)
#print(f"Encoded automobile data : \n{df_encoded}")

In [20]:
df_encoded

,mpg,cyl,displ,hp,weight,accel,year,origin_enc,name_amc ambassador brougham,name_amc ambassador dpl,...,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
0,18.0,8,307.0,130,3504,12.0,1970,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,8,350.0,165,3693,11.5,1970,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,18.0,8,318.0,150,3436,11.0,1970,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16.0,8,304.0,150,3433,12.0,1970,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17.0,8,302.0,140,3449,10.5,1970,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,1982,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,44.0,4,97.0,52,2130,24.6,1982,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
389,32.0,4,135.0,84,2295,11.6,1982,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
390,28.0,4,120.0,79,2625,18.6,1982,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Linear Regression

In [21]:
X = df_encoded.drop('mpg', axis=1)
y = df_encoded['mpg']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [22]:
lin_reg.score(X_test, y_test)

0.7016636103435525

In [37]:
X_test

,cyl,displ,hp,weight,accel,year,origin_enc,name_amc ambassador brougham,name_amc ambassador dpl,name_amc ambassador sst,...,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
78,4,96.0,69,2189,18.0,1972,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,4,121.0,115,2795,15.7,1978,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
246,4,91.0,60,1800,16.4,1978,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,4,91.0,70,1955,20.5,1971,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,4,140.0,86,2790,15.6,1982,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,6,225.0,85,3465,16.6,1981,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,4,98.0,80,2164,15.0,1972,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,8,350.0,145,4082,13.0,1973,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,304.0,150,3433,12.0,1970,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
lr_pred = lin_reg.predict(X_test)
lr_pred

array([26.67427937, 26.29334183, 35.24826332, 25.25803272, 30.15040624,
       29.99006384,  6.57827848, 29.55634711, 21.02971714, 29.10663841,
       13.55453712, 20.73523677, 12.35870939, 29.49174804, 20.34507063,
       28.44642524, 22.19620464, 31.95544821, 29.43583422, 31.08890238,
       21.7613948 , 30.35187468, 35.4748409 , 14.65937608, 28.01853   ,
       27.35881219, 20.40159196, 17.19332243, 28.0560938 , 24.62018025,
       12.05145522, 22.47437824, 19.340346  , 30.97090914, 11.70803382,
       35.9981724 ,  8.93571437, 26.61338989,  9.62293513,  4.20886672,
       12.55083586, 29.64419535, 35.89240967, 24.16038007, 12.81197612,
        7.8908827 , 18.84743296, 33.5962462 , 22.69997466, 40.65164701,
       12.89241638, 26.96066819, 25.86796673, 35.2078005 , 28.05989181,
       15.33999683, 20.62485661, 19.30460053, 23.30631484, 24.60602788,
        6.22530732, 18.95226184, 27.42759923, 26.62411153, 30.08685832,
       29.37572837, 27.96332182, 27.08632436, 26.67505508, 11.53

### Lasso Regression

In [23]:
lasso = Lasso(alpha=0.5)
lasso.fit(X_train, y_train)

Lasso(alpha=0.5)

In [24]:
lasso.score(X_test, y_test)

0.7933042014150093

In [40]:
X_test

,cyl,displ,hp,weight,accel,year,origin_enc,name_amc ambassador brougham,name_amc ambassador dpl,name_amc ambassador sst,...,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
78,4,96.0,69,2189,18.0,1972,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,4,121.0,115,2795,15.7,1978,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
246,4,91.0,60,1800,16.4,1978,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,4,91.0,70,1955,20.5,1971,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,4,140.0,86,2790,15.6,1982,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,6,225.0,85,3465,16.6,1981,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,4,98.0,80,2164,15.0,1972,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,8,350.0,145,4082,13.0,1973,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,304.0,150,3433,12.0,1970,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
lasso_pred = lasso.predict(X_test)

### Ridge Regression

In [25]:
ridge = Ridge(alpha=0.5)
ridge.fit(X_train, y_train)

Ridge(alpha=0.5)

In [26]:
ridge.score(X_test, y_test)

0.7699206299439977

In [41]:
X_test

,cyl,displ,hp,weight,accel,year,origin_enc,name_amc ambassador brougham,name_amc ambassador dpl,name_amc ambassador sst,...,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
78,4,96.0,69,2189,18.0,1972,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,4,121.0,115,2795,15.7,1978,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
246,4,91.0,60,1800,16.4,1978,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,4,91.0,70,1955,20.5,1971,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,4,140.0,86,2790,15.6,1982,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,6,225.0,85,3465,16.6,1981,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,4,98.0,80,2164,15.0,1972,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,8,350.0,145,4082,13.0,1973,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,304.0,150,3433,12.0,1970,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
ridge_pred = ridge.predict(X_test)
ridge_pred

array([26.0994572 , 25.7488422 , 34.94011955, 24.90840053, 28.56725127,
       29.94862448,  6.42433617, 30.53362344, 20.86606993, 28.96497458,
       12.74100052, 22.90019273, 14.50101592, 29.16861213, 21.47792545,
       28.76910827, 21.89038819, 32.53046678, 27.14233074, 30.06302838,
       20.69586857, 33.62529903, 35.58331866, 15.06056026, 28.06651536,
       25.92258853, 20.82482168, 16.57996398, 28.76984875, 24.22907878,
       13.35843381, 22.86046801, 20.85254949, 31.46387287, 11.25090859,
       36.26816659, 10.77312915, 24.74474647, 11.37269239,  6.19198465,
       13.21945429, 28.04782358, 34.92111256, 24.7617955 , 12.67967954,
        9.04603331, 18.08495622, 31.90394511, 23.41994141, 36.95449669,
       12.68043801, 26.58403259, 23.53727926, 35.07545575, 28.17645552,
       16.18509277, 20.89754605, 20.83317665, 22.72173172, 24.40840917,
        7.57278301, 20.40170659, 26.14101807, 24.35344303, 29.90430959,
       30.30374986, 26.01024186, 28.60940463, 23.02627646, 10.57

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report

In [32]:
random_forest = RandomForestRegressor()
random_forest.fit(X_train, y_train)

RandomForestRegressor()

In [33]:
random_forest.score(X_test, y_test)

0.8894160745752522

In [43]:
X_test

,cyl,displ,hp,weight,accel,year,origin_enc,name_amc ambassador brougham,name_amc ambassador dpl,name_amc ambassador sst,...,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
78,4,96.0,69,2189,18.0,1972,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,4,121.0,115,2795,15.7,1978,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
246,4,91.0,60,1800,16.4,1978,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,4,91.0,70,1955,20.5,1971,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,4,140.0,86,2790,15.6,1982,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,6,225.0,85,3465,16.6,1981,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,4,98.0,80,2164,15.0,1972,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,8,350.0,145,4082,13.0,1973,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,304.0,150,3433,12.0,1970,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
rf_pred = random_forest.predict(X_test)
rf_pred

array([26.441, 22.612, 34.473, 29.861, 27.222, 28.278, 12.75 , 27.368,
       19.563, 31.218, 14.215, 20.033, 15.92 , 27.231, 19.168, 29.402,
       20.425, 32.974, 28.772, 30.768, 21.88 , 38.116, 33.844, 15.937,
       30.961, 23.9  , 19.665, 19.159, 28.902, 23.56 , 14.42 , 19.576,
       20.01 , 30.842, 13.54 , 34.598, 14.19 , 27.485, 14.745, 11.76 ,
       15.032, 29.011, 34.04 , 26.01 , 14.425, 11.76 , 17.933, 34.003,
       22.998, 30.884, 14.075, 23.374, 23.78 , 33.644, 26.062, 16.141,
       18.272, 18.882, 24.11 , 24.683, 11.86 , 19.529, 25.281, 20.554,
       25.787, 26.081, 24.823, 30.819, 19.747, 14.05 , 23.79 , 13.6  ,
       21.597, 27.297, 22.957, 26.969, 13.69 , 15.234, 26.96 ])

In [34]:
from sklearn.tree import DecisionTreeRegressor

In [35]:
dec_tree = DecisionTreeRegressor()
dec_tree.fit(X_train, y_train)

DecisionTreeRegressor()

In [36]:
dec_tree.score(X_test, y_test)

0.8784337926268838

In [45]:
X_test

,cyl,displ,hp,weight,accel,year,origin_enc,name_amc ambassador brougham,name_amc ambassador dpl,name_amc ambassador sst,...,name_volvo 145e (sw),name_volvo 244dl,name_volvo 245,name_volvo 264gl,name_volvo diesel,name_vw dasher (diesel),name_vw pickup,name_vw rabbit,name_vw rabbit c (diesel),name_vw rabbit custom
78,4,96.0,69,2189,18.0,1972,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,4,121.0,115,2795,15.7,1978,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
246,4,91.0,60,1800,16.4,1978,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,4,91.0,70,1955,20.5,1971,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,4,140.0,86,2790,15.6,1982,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,6,225.0,85,3465,16.6,1981,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,4,98.0,80,2164,15.0,1972,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,8,350.0,145,4082,13.0,1973,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,304.0,150,3433,12.0,1970,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
dt_pred = dec_tree.predict(X_test)
dt_pred

array([23. , 19. , 38.1, 30. , 24. , 24. , 14. , 30.9, 18. , 26. , 15. ,
       20.3, 16. , 24. , 17. , 27. , 17. , 32.4, 27.2, 33.5, 23. , 36. ,
       39. , 15. , 31. , 28. , 19. , 18. , 26. , 23. , 16. , 20.8, 23.9,
       31.3, 13. , 37. , 10. , 23. , 14. , 11. , 14. , 24. , 36. , 26. ,
       16. , 11. , 18. , 43.4, 25. , 30. , 14. , 22. , 23. , 33.5, 21.5,
       16. , 18. , 18. , 24. , 23. , 11. , 22. , 28.4, 23.8, 24. , 23. ,
       25. , 30. , 23.9, 14. , 23. , 13. , 22. , 19. , 19.1, 26. , 13. ,
       15. , 24. ])